In [1]:
# !pip install yfinance

In [2]:
import pandas as pd
import yfinance as yf

In [3]:
TICKERS = ['MSFT','GOOG','META']

# Historical Data

In [4]:
def yf_historical_data(TICKERS, PERIOD='1d', UNTIL='max'):
    Pipe = []
    for i, TICKER in enumerate(TICKERS):
        try: 
            Data = yf.download(TICKER,  interval=PERIOD, period=UNTIL, ignore_tz=True,  auto_adjust=False, back_adjust=False,  multi_level_index=False).reset_index()
            Data.insert(0, 'Ticker', TICKER)
            Pipe.append(Data)
        except Exception as E: 
            print(i, TICKER, E)
    pass
    return pd.concat(Pipe, ignore_index=True)

Data = yf_historical_data(TICKERS)
Data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Ticker,Date,Adj Close,Close,High,Low,Open,Volume
0,MSFT,1986-03-13,0.059827,0.097222,0.101563,0.088542,0.088542,1031788800
1,MSFT,1986-03-14,0.061963,0.100694,0.102431,0.097222,0.097222,308160000
2,MSFT,1986-03-17,0.063032,0.102431,0.103299,0.100694,0.100694,133171200
3,MSFT,1986-03-18,0.061429,0.099826,0.103299,0.098958,0.102431,67766400
4,MSFT,1986-03-19,0.060361,0.098090,0.100694,0.097222,0.099826,47894400
...,...,...,...,...,...,...,...,...
18178,META,2025-02-14,736.669983,736.669983,740.909973,725.619995,726.140015,16901200
18179,META,2025-02-18,716.369995,716.369995,737.000000,706.440002,736.000000,21938700
18180,META,2025-02-19,703.770020,703.770020,707.000000,695.390015,704.099976,17535500
18181,META,2025-02-20,694.840027,694.840027,705.630005,693.640015,697.280029,12587400


# Quote

In [5]:
def yf_quote(TICKERS):
    pipe = []
    for i, TICKER in enumerate(TICKERS):
        try: 
            info = yf.Ticker(TICKER).info
            pipe.append({
                'Ticker': TICKER, 
                'Price':  info['currentPrice'], 
                'Prev':   info['previousClose'], 
                'Ask':    info['ask'], 
                'Bid':    info['bid'], 
            })
        except Exception as E: 
            print(i, TICKER, E)
    pass
    return pd.DataFrame(pipe)

Quote = yf_quote(TICKERS)
Quote

,Ticker,Price,Prev,Ask,Bid
0,MSFT,408.21,416.13,408.39,408.23
1,GOOG,181.58,186.64,192.98,174.18
2,META,683.55,694.84,683.74,682.91


# Options

In [6]:
def yf_options_chain(TICKERS):
    Pipe = []
    for i, TICKER in enumerate(TICKERS):
        try: 
            Asset       = yf.Ticker(TICKER)
            expirations = Asset.options
            for j, EXPIRY in enumerate(expirations):
                try: 
                    for k, OPTION in enumerate(['calls','puts']):
                        try: 
                            Chain = getattr(Asset.option_chain(EXPIRY), OPTION)
                            Chain.insert(0, 'Ticker', TICKER)
                            Chain.insert(1, 'Option', OPTION[0].capitalize())
                            Chain.insert(2, 'Expiry', EXPIRY)
                            Pipe.append(Chain)
                        except Exception as E:  print(i, TICKER, j, EXPIRY, k, OPTION, E)
                    pass
                except Exception as E:  print(i, TICKER, j, EXPIRY, E)
            pass
        except Exception as E:  print(i, TICKER, E)
    pass
    return pd.concat(Pipe, ignore_index=True)

OChain = yf_options_chain(TICKERS)
OChain

,Ticker,Option,Expiry,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MSFT,C,2025-02-28,MSFT250228C00240000,2025-02-12 15:08:15+00:00,240.0,167.72,166.45,170.50,0.000000,0.000000,NaN,0.0,1.802735,True,REGULAR,USD
1,MSFT,C,2025-02-28,MSFT250228C00270000,2025-02-06 16:30:58+00:00,270.0,147.30,136.45,140.50,0.000000,0.000000,1.0,0.0,1.437503,True,REGULAR,USD
2,MSFT,C,2025-02-28,MSFT250228C00300000,2025-01-31 19:55:07+00:00,300.0,117.17,106.50,110.75,0.000000,0.000000,7.0,7.0,1.183598,True,REGULAR,USD
3,MSFT,C,2025-02-28,MSFT250228C00320000,2025-02-20 14:51:42+00:00,320.0,97.07,86.50,91.00,0.000000,0.000000,3.0,6.0,1.008794,True,REGULAR,USD
4,MSFT,C,2025-02-28,MSFT250228C00325000,2025-02-21 18:57:05+00:00,325.0,86.60,81.50,85.50,-3.099998,-3.455963,1.0,12.0,0.862306,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8075,META,P,2027-01-15,META270115P01170000,2025-02-05 18:31:51+00:00,1170.0,466.58,484.55,488.50,0.000000,0.000000,NaN,0.0,0.198708,True,REGULAR,USD
8076,META,P,2027-01-15,META270115P01180000,2025-02-05 18:31:51+00:00,1180.0,476.43,494.50,498.75,0.000000,0.000000,NaN,0.0,0.204689,True,REGULAR,USD
8077,META,P,2027-01-15,META270115P01190000,2025-02-18 18:04:27+00:00,1190.0,474.40,504.45,508.75,0.000000,0.000000,1.0,0.0,0.207222,True,REGULAR,USD
8078,META,P,2027-01-15,META270115P01250000,2025-01-22 16:03:51+00:00,1250.0,622.10,564.50,568.80,0.000000,0.000000,2.0,0.0,0.222542,True,REGULAR,USD
